In [3]:
import geopandas as gpd
from shapely.geometry import Point, LineString
from shapely.ops import nearest_points
import pandas as pd
import folium

# Laad de gebruikersprofielen en routes
user_profiles_path = '../Data/Loopdata/kleinebestanden/run_gebruikersprofielen.json'
routes_path = '../Data/Loopdata/kleinebestanden/routes_volledig.json'
user_profiles_df = pd.read_json(user_profiles_path, lines=True)
routes_df = pd.read_json(routes_path, lines=True)


# Aannemend dat routes_df en user_profiles_df al gedefinieerd zijn
gdf_routes = gpd.GeoDataFrame(routes_df, geometry=gpd.points_from_xy(routes_df.longitude, routes_df.latitude))
gdf_user_profiles = gpd.GeoDataFrame(user_profiles_df, geometry=gpd.points_from_xy(user_profiles_df.most_common_lon, user_profiles_df.most_common_lat))
sindex_routes = gdf_routes.sindex

def find_nearby_segments(user_location, max_distance, sindex, gdf):
    nearby_indices = list(sindex.intersection(user_location.buffer(max_distance).bounds))
    nearby_segments = gdf.iloc[nearby_indices]
    return nearby_segments

def generate_new_route(user_id, gdf_user_profiles, gdf_routes, max_distance=10):
    user_profile = gdf_user_profiles[gdf_user_profiles['userId'] == user_id].iloc[0]
    user_location = user_profile.geometry
    
    nearby_segments = find_nearby_segments(user_location, max_distance, sindex_routes, gdf_routes)
    
    # Sorteer segmenten op basis van afstand tot gebruiker
    sorted_segments = nearby_segments.copy()
    sorted_segments['distance_to_user'] = sorted_segments.apply(lambda x: user_location.distance(x.geometry), axis=1)
    sorted_segments = sorted_segments.sort_values(by='distance_to_user')

    # Bouw de route op
    new_route_segments = []
    current_location = user_location
    remaining_distance = max_distance

    for _, segment in sorted_segments.iterrows():
        # Bepaal het dichtstbijzijnde punt op het segment tot de huidige locatie
        nearest_point = nearest_points(current_location, segment.geometry)[1]

        # Controleer of het segment binnen de resterende afstand past
        segment_distance = segment['distance']
        if segment_distance <= remaining_distance:
            new_route_segments.append(segment)
            current_location = Point(segment.geometry.coords[-1])
            remaining_distance -= segment_distance
            if remaining_distance <= 0:
                break

    return gpd.GeoDataFrame(pd.concat(new_route_segments, ignore_index=True))

# Test de functie voor een specifieke gebruiker
sample_user_id = 367324
new_route = generate_new_route(sample_user_id, gdf_user_profiles, gdf_routes)

# Verdere verwerking of visualisatie van new_route
# Zoek het gebruikersprofiel voor visualisatie
user_profile = user_profiles_df[user_profiles_df['userId'] == sample_user_id].iloc[0]

# Visualiseer de nieuwe route
map_obj = folium.Map(location=[user_profile['most_common_lat'], user_profile['most_common_lon']], zoom_start=12)
popup_text = f"UserID: {sample_user_id}<br>Average Speed: {user_profile['average_speeds']}<br>Average Altitude Change: {user_profile['average_altitude_changes']}<br>Average Distance: {user_profile['average_distances']}<br>Average Condition Value: {user_profile['average_condition_value']}"
folium.Marker(location=[user_profile['most_common_lat'], user_profile['most_common_lon']], popup=popup_text).add_to(map_obj)

for _, segment in new_route.iterrows():
    segment_coords = list(zip(segment['latitude'], segment['longitude']))
    folium.PolyLine(segment_coords, color='blue', weight=2.5, opacity=1).add_to(map_obj)

map_obj

ValueError: setting an array element with a sequence.